# But du projet

Ce code utilise l'API Roboflow pour télécharger la dataset de 3 personnages de dragon ball :
- Goku
- Vegeta
- Freezer

Executer cette commande pour installer la bibliothèque roboflow
```
pip install roboflow
```

# Installation des bibliothèques nécessaires

In [24]:
# Tous les imports nécessaires
import shutil
from roboflow import Roboflow
import os
import random
import cv2
from ultralytics import YOLO, settings
import torch
from moviepy.editor import VideoFileClip, ImageSequenceClip
import sys
import stat

settings.update(datasets_dir='.') # on definit le dossier de sauvegarde des datasets


# Téléchargement du dataset via l'API Roboflow si elle n'est pas déjà présente

In [25]:
# Exemple pour modifier les permissions du répertoire courant
current_dir = os.getcwd()

# Donner des permissions d'écriture au répertoire courant (777 donne des permissions complètes)
os.chmod(current_dir, stat.S_IRWXU | stat.S_IRWXG | stat.S_IRWXO)

# Initialiser l'objet Roboflow avec une clé API
rf = Roboflow(api_key="aVyMuIoKSec5dyUaxD6H")

# Accéder à un projet spécifique dans un espace de travail
project = rf.workspace("converter-sq0eo").project("dbf-3hywn")

# Sélectionner une version spécifique du projet
version = project.version(5)

# Télécharger le dataset au format YOLOv8
def download_dataset():
    dataset = version.download("yolov8")

# Vérifier si un dossier contient les chaînes "dbf" et "yolo"
def check_dataset_directory():
    
    # Récupérer le répertoire courant
    current_dir = os.getcwd()
    
    # Parcourir les dossiers du répertoire courant
    for dir_name in os.listdir(current_dir):
        if "dbf" in dir_name.lower() and "5" in dir_name.lower():
            dataset_dir = os.path.join(current_dir, dir_name)
            if os.path.isdir(dataset_dir):
                required_dirs = ["test", "train", "valid"]
                required_files = ["data.yaml", "README.dataset.txt", "README.roboflow.txt"]
                for req_dir in required_dirs:
                    if not os.path.isdir(os.path.join(dataset_dir, req_dir)):
                        shutil.rmtree(dataset_dir)
                        return False
                for req_file in required_files:
                    if not os.path.isfile(os.path.join(dataset_dir, req_file)):
                        shutil.rmtree(dataset_dir)
                        return False
                return True
    return False

# Vérifier et télécharger si nécessaire
if not check_dataset_directory():
    download_dataset()
else:
    print("Le dataset est déjà présent et complet.")

loading Roboflow workspace...
loading Roboflow project...
Le dataset est déjà présent et complet.


# Entraînement du modèle YOLOv8 si ça n'a pas déjà été fait

Si ce n'est pas déjà fait, l'entrainement durerait en moyenne 30 minutes avec un GPU, alors que sans GPU, cela prendrait plus longtemps.

In [27]:
# Définir le périphérique en fonction de la disponibilité du GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Utilisation du périphérique: {device}")

# Charger le modèle YOLOv8 et déplacer sur le GPU si disponible

name_file = "yolov8n-final.pt"

model = None

if not os.path.exists(name_file):
    model = YOLO('yolov8n.pt').to(device)
    # Charger les données d'entraînement
    data_yaml_path = 'DBF-5/data.yaml'

    # Vérifier si le fichier data.yaml existe
    if not os.path.exists(data_yaml_path):
        raise FileNotFoundError(f"Le fichier {data_yaml_path} est introuvable.")

    print("Chargement des données d'entraînement...")
    try:
        model.train(data=data_yaml_path, epochs=10, device=device, batch=8)
        print("Entraînement terminé.")
        model.save(name_file)
        print("Model sauvegardé avec succès.")
    except Exception as e:
        print(f"Erreur lors de l'entraînement du modèle: {e}")
else:
    print("Model déjà existant")
    model = YOLO(name_file).to(device)

Utilisation du périphérique: cpu
Model déjà existant


# Utilisation du modèle YOLOv8 pour la détection en temps réel

In [ ]:
# Sélectionner un fichier MP4 aléatoire dans le dossier mp4
mp4_dir = 'mp4'
mp4_files = [f for f in os.listdir(mp4_dir) if f.endswith('.mp4')]
random_mp4 = random.choice(mp4_files)
video_path = os.path.join(mp4_dir, random_mp4)

print(f"Utilisation du fichier vidéo: {video_path}")

# Charger la vidéo avec OpenCV
cap = cv2.VideoCapture(video_path)

# Obtenir les propriétés de la vidéo
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Créer un chemin de sortie pour la vidéo
output_video_path = video_path.replace(".mp4", "_with_detection.mp4")

# Initialiser un writer pour écrire la vidéo avec détection
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# Lire et traiter chaque frame
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Effectuer la détection sur la frame
    results = model(frame)  # Supposons que `model` renvoie des résultats

    # Traiter les résultats de la détection
    for result in results:
        for box in result.boxes:
            try:
                x1, y1, x2, y2 = box.xyxy[0]
                label = result.names[int(box.cls)]
                confidence = box.conf[0]
            except (ValueError, IndexError, AttributeError) as e:
                print(f"Erreur lors de la lecture des coordonnées: {e}")
                continue

            # Dessiner le rectangle en rouge
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 2)

            # Ajouter le label et la confiance au-dessous du rectangle
            cv2.putText(frame, f'{label} {confidence:.2f}', (int(x1), int(y1) + 20),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

    # Écrire la frame avec les détections dans la vidéo de sortie
    out.write(frame)

# Libérer les ressources
cap.release()
out.release()

print(f"Vidéo avec détection enregistrée : {output_video_path}")

# Fusionner l'audio d'origine avec la vidéo modifiée
original_video = VideoFileClip(video_path)
video_with_detection = VideoFileClip(output_video_path)

# Remplacer l'audio dans la vidéo modifiée par celui de la vidéo originale
final_video_path = output_video_path.replace("_with_detection.mp4", "_with_detection_with_audio.mp4")
video_with_audio = video_with_detection.set_audio(original_video.audio)
video_with_audio.write_videofile(final_video_path, codec="libx264", audio_codec="aac")

print(f"Vidéo finale enregistrée avec audio : {final_video_path}")

# Supprimer le fichier vidéo sans audio
if os.path.exists(output_video_path):
    os.remove(output_video_path)
    print(f"Fichier temporaire sans audio supprimé : {output_video_path}")
sys.exit()

Utilisation du fichier vidéo: mp4\DBS_vegeta_ssj_bleu_vs_golden_freezer_vf.mp4

0: 384x640 (no detections), 128.5ms
Speed: 6.8ms preprocess, 128.5ms inference, 5.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 82.4ms
Speed: 3.0ms preprocess, 82.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 78.7ms
Speed: 1.0ms preprocess, 78.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 77.5ms
Speed: 2.0ms preprocess, 77.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 73.1ms
Speed: 2.0ms preprocess, 73.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 74.4ms
Speed: 2.0ms preprocess, 74.4ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 81.3ms
Speed: 1.6ms preprocess, 81.3ms inference, 1.0ms postprocess per image at shape (1, 3